In [5]:
import os
import glob
from osgeo import gdal
import rasterio

In [10]:
def create_vrt_and_convert_to_cog_2(input_folder, output_folder, test_mode=False):
    os.makedirs(output_folder, exist_ok=True)
    intermediate_file = os.path.join(output_folder, "merged.tif")
    output_cog_path = os.path.join(output_folder, "final_output_cog.tif")
    input_files = glob.glob(os.path.join(input_folder, "**", "*.tif"), recursive=True)

    if test_mode:
        input_files = input_files[:10]  # Process only the first 10 files in test mode

    # Get the CRS from the first input file
    with rasterio.open(input_files[0]) as src:
        src_crs = src.crs

    print("Merging input files...")
    gdal.Warp(intermediate_file, input_files, options=gdal.WarpOptions(srcSRS=src_crs, dstSRS=src_crs))

    intermediate_ds = gdal.Open(intermediate_file)
    if intermediate_ds is None:
        print("Error: Could not open intermediate file.")
        return None

    print(f"Source CRS: {src_crs}")

    gdal.SetConfigOption('GDAL_CACHEMAX', '1024')
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')
    
    translate_options = gdal.TranslateOptions(
        format='COG',
        creationOptions=[
            "COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES",
            "BLOCKSIZE=512", "OVERVIEW_RESAMPLING=AVERAGE", "NUM_THREADS=ALL_CPUS"
        ],
        outputSRS=src_crs
    )
    
    print("Converting to COG...")
    gdal.Translate(output_cog_path, intermediate_file, options=translate_options)
    os.remove(intermediate_file)

    output_ds = gdal.Open(output_cog_path)
    if output_ds is None:
        print("Error: Could not open output COG file.")
        return None

    output_crs = output_ds.GetProjection()
    output_ds = None

    if src_crs.to_string() == output_crs:
        print("CRS preserved successfully.")
    else:
        print("Warning: CRS may not have been preserved.")
        print(f"Source CRS: {src_crs}")
        print(f"Output CRS: {output_crs}")

    return output_cog_path

In [11]:
# Example usage
input_folder = '/Users/shuyang/Data/DTM/Sudbury/Sudbury-DTM-11/DTM'
output_folder = '//Users/shuyang/Data/DTM/Sudbury/Sudbury-DTM-11'
create_vrt_and_convert_to_cog_2(input_folder, output_folder,False)

Merging input files...
Source CRS: EPSG:2958
Converting to COG...
Source CRS: EPSG:2958
Output CRS: PROJCS["NAD83(CSRS) / UTM zone 17N",GEOGCS["NAD83(CSRS)",DATUM["NAD83_Canadian_Spatial_Reference_System",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6140"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4617"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2958"]]


'//Users/shuyang/Data/DTM/Sudbury/Sudbury-DTM-11/final_output_cog.tif'

In [8]:
def compare_spatial_reference(output_raster, original_dtm):
    with rasterio.open(output_raster) as out_src, rasterio.open(original_dtm) as orig_src:
        print("Output Raster:")
        print(f"CRS: {out_src.crs}")
        print(f"Transform: {out_src.transform}")
        print(f"Bounds: {out_src.bounds}")
        print("\nOriginal DTM:")
        print(f"CRS: {orig_src.crs}")
        print(f"Transform: {orig_src.transform}")
        print(f"Bounds: {orig_src.bounds}")
        
        if out_src.crs == orig_src.crs:
            print("\nCRS matches between output and original.")
        else:
            print("\nWarning: CRS mismatch between output and original.")
        
        if out_src.transform.almost_equals(orig_src.transform):
            print("Transform matches between output and original.")
        else:
            print("Warning: Transform mismatch between output and original.")



In [9]:
# After creating the COG, compare it with an original DTM file
output_cog = create_vrt_and_convert_to_cog_2(input_folder, output_folder, True)

if output_cog:
    # Assuming the first file in input_folder is a representative original DTM
    original_dtm = os.path.join(input_folder, os.listdir(input_folder)[0])
    compare_spatial_reference(output_cog, original_dtm)
else:
    print("COG creation failed, no comparison performed.")

Merging input files...
Source CRS: EPSG:2958
Converting to COG...
Source CRS: EPSG:2958
Output CRS: PROJCS["NAD83(CSRS) / UTM zone 17N",GEOGCS["NAD83(CSRS)",DATUM["NAD83_Canadian_Spatial_Reference_System",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6140"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4617"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2958"]]
Output Raster:
CRS: EPSG:2958
Transform: | 0.50, 0.00, 484000.00|
| 0.00,-0.50, 5191000.00|
| 0.00, 0.00, 1.00|
Bounds: BoundingBox(left=484000.0, bottom=5168000.0, right=497000.0, top=5191000.0)

Original DTM:
CRS: EPSG:2958
Transform: | 0.50, 0.00, 493000.0

In [ ]:
def legacy_create_vrt_and_convert_to_cog(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    vrt_path = os.path.join(output_folder, "merged.vrt")
    output_cog_path = os.path.join(output_folder, "final_output_cog.tif")
    input_files = glob.glob(os.path.join(input_folder, "**", "*.tif"), recursive=True)

    vrt = gdal.BuildVRT(vrt_path, input_files, options=gdal.BuildVRTOptions(resampleAlg='nearest', addAlpha=False))
    
    # Get the CRS from the VRT
    src_crs = vrt.GetProjection()
    vrt = None

    gdal.SetConfigOption('GDAL_CACHEMAX', '1024')
    gdal.SetConfigOption('GDAL_NUM_THREADS', 'ALL_CPUS')
    
    translate_options = gdal.TranslateOptions(
        format='COG',
        creationOptions=[
            "COMPRESS=LZW", "PREDICTOR=2", "BIGTIFF=YES",
            "BLOCKSIZE=512", "OVERVIEW_RESAMPLING=AVERAGE", "NUM_THREADS=ALL_CPUS"
        ],
        outputSRS=src_crs  # Set the output SRS to match the source
    )
    
    gdal.Translate(output_cog_path, vrt_path, options=translate_options)
    os.remove(vrt_path)

    # Verify the CRS of the output COG
    output_ds = gdal.Open(output_cog_path)
    output_crs = output_ds.GetProjection()
    output_ds = None

    if src_crs == output_crs:
        print("CRS preserved successfully.")
    else:
        print("Warning: CRS may not have been preserved.")

    return output_cog_path

In [ ]:
def upload_to_aws(local_file, bucket, s3_folder):
    s3 = boto3.client('s3')
    
    try:
        file_name = os.path.basename(local_file)
        s3_file = os.path.join(s3_folder, file_name)
        
        print(f"Uploading {file_name} to AWS S3...")
        s3.upload_file(local_file, bucket, s3_file)
        print(f"Upload Successful. File available at: https://{bucket}.s3.amazonaws.com/{s3_file}")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [ ]:
# Upload to S3
if cog_file:
    upload_to_aws(cog_file, 'ccemp-bucket', 'Public Access/COG/')
else:
    print("COG creation failed, no upload attempted.")